## Notebook 4.3: ReAct

In [ ]:
from llama_stack_client import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
import sys
sys.path.append('.') 
from src.client_tools import get_location

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](./Level1_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [ ]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv(override=True)

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# pretty print of the results returned from the model/agent
import sys
sys.path.append('.')  
from src.utils import step_printer
from termcolor import cprint

base_url = os.getenv("REMOTE_BASE_URL")


client = LlamaStackClient(base_url=base_url)
    
print(f"Connected to Llama Stack server")

# model_id for the model you wish to use that is configured with the Llama Stack server
model_id = "llama3-2-3b" # "deepseek-r1-0528-qwen3-8b-bnb-4bit"

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = 5000

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream = "True"

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

### ReAct Agent with websearch tool and client tool

This section demonstrates the ReAct (Reasoning and Acting) framework in action.

Here is a walkthrough of how the ReAct agent will tackle this same "weather near me" problem:

When asked "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent will:

1. **Reason** that it needs to get location information first.
2. **Act** by calling the `get_location` client tool.
3. **Observe** the location result.
4. **Reason** that it now needs to search for weather in that location.
5. **Act** by calling the `websearch` tool with observed location.
6. **Observe** and processes the search results into a final answer. 

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

We are going to switch LLM models here from the LLM llama32-3b model to the more powerful reasoning model deepseek-r1-0528-qwen3-8b-bnb-4bit that is able to more accurately reason about the tools it will use. Resoning models output their \<thought\> process first.

In [ ]:
model_id = "deepseek-r1-0528-qwen3-8b-bnb-4bit" # "llama3-2-3b"

agent = ReActAgent(
            client=client,
            model=model_id,
            tools=[get_location, "builtin::websearch"],
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            sampling_params=sampling_params,
        )
user_prompts = [
    "Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?"
]
session_id = agent.create_session("web-session")
for prompt in user_prompts:
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 

## Key Takeaways
- This notebook demonstrated how to build more capable agents using Prompt Chaining and the ReAct framework.
- It showed how agents can maintain context across multiple steps and perform structured, multi-step reasoning.
- It highlights how ReAct enables dynamic tool selection and adaptive decision-making based on intermediate results.
- These techniques enhance agent autonomy and make them more suitable for complex operational tasks.

For further extensions, continue exploring in the next notebook: [Agents and MCP](5_mcp.ipynb).